<a href="https://colab.research.google.com/github/harshari/PlantAI/blob/main/MIDI_to_notes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install mido

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.8 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


In [ ]:
import mido

def midi_to_notes(midi_file, default_tempo=500000):  # Default tempo: 120 bpm
    midi = mido.MidiFile(midi_file)
    notes = []
    tempo = default_tempo  # Microseconds per beat
    for track in midi.tracks:
        current_time = 0  # Reset time for each track
        for msg in track:
            if msg.type == 'set_tempo':
                tempo = msg.tempo
            if msg.type == 'note_on' and msg.velocity > 0:
                note_start_time = current_time
                note = msg.note

                # Accumulate time until the note off event
                for follow_msg in track:
                    if follow_msg.type == 'note_off' or (follow_msg.type == 'note_on' and follow_msg.velocity == 0 and follow_msg.note == note):
                        note_end_time = note_start_time + follow_msg.time
                        duration = mido.tick2second(note_end_time - note_start_time, midi.ticks_per_beat, tempo)
                        notes.append((note, msg.velocity, round(duration * 1000)))  # Duration in milliseconds
                        break
                    note_start_time += follow_msg.time  # Update time until note off

            # Update current time for the next message
            current_time += msg.time

    return notes

# Example usage
midi_path = '/content/Queen_midi.midi'
notes = midi_to_notes(midi_path)

notes_string = '\n'.join([f'{note},{velocity},{duration}' for note, velocity, duration in notes])

# Write to a text file
with open('queen_notes.txt', 'w') as file:
    file.write(notes_string)

# Print the notes
for note in notes:
    print(note)


# Only issue of the above process is that it loses the sense of chords, as the information about the notes which are played together is lost. How can we capture that??


### A good way might be to also have chords as different MIDI channel

In [ ]:
import mido

def midi_to_notes(midi_file, default_tempo=500000, chord_threshold=50):  # Default tempo: 120 bpm
    midi = mido.MidiFile(midi_file)
    events = []
    tempo = default_tempo
    current_time = 0

    for track in midi.tracks:
        for msg in track:
            current_time += mido.tick2second(msg.time, midi.ticks_per_beat, tempo) * 1000  # Convert to milliseconds

            if msg.type == 'note_on' and msg.velocity > 0:
                events.append(('on', msg.note, current_time, msg.velocity))
            elif msg.type == 'note_off' or (msg.type == 'note_on' and msg.velocity == 0):
                # Note 'off' events don't have a velocity, so use a placeholder (e.g., 0)
                events.append(('off', msg.note, current_time, 0))

    # Sort events by time
    events.sort(key=lambda x: x[2])

    # Process events to form chords
    chords = []
    current_chord = []
    last_time = 0
    last_velocity = 0

    for event in events:
        event_type, note, time, velocity = event

        if event_type == 'on':
            if current_chord and time - last_time > chord_threshold:
                # Finalize the current chord
                duration = last_time - current_chord[0][2]
                chord_notes = [n for t, n, tm, v in current_chord]
                chords.append((chord_notes, last_velocity, duration))
                current_chord = []

            current_chord.append(event)
            last_time = time
            last_velocity = velocity

        elif event_type == 'off':
            if current_chord:
                current_chord = [e for e in current_chord if e[1] != note]

    # Add the last chord if any
    if current_chord:
        duration = last_time - current_chord[0][2]
        chord_notes = [n for t, n, tm, v in current_chord]
        chords.append((chord_notes, last_velocity, duration))

    return chords

# Example usage
midi_path = '/content/Queen_midi.midi'
chords = midi_to_notes(midi_path)

# Convert chords to a string format and save to a file
chords_string = '\n'.join([','.join(map(str, chord[0])) + ',' + str(chord[1]) + ',' + str(chord[2]) for chord in chords])

with open('midi_chords_formatted.txt', 'w') as file:
    file.write(chords_string)

# Print the chords for review
for chord in chords:
    print(chord)


In [16]:
import mido

def midi_to_notes(midi_file, default_tempo=500000, chord_threshold=50):  # Default tempo: 120 bpm
    midi = mido.MidiFile(midi_file)
    note_start_times = {}
    chords = []
    current_chord = []
    current_time = 0
    tempo = default_tempo

    for track in midi.tracks:
        for msg in track:
            current_time += mido.tick2second(msg.time, midi.ticks_per_beat, tempo) * 1000  # Convert to milliseconds

            if msg.type == 'note_on' and msg.velocity > 0:
                note_start_times[msg.note] = current_time  # Note start time
                current_chord.append((msg.note, msg.velocity))

            elif (msg.type == 'note_off' or (msg.type == 'note_on' and msg.velocity == 0)) and msg.note in note_start_times:
                start_time = note_start_times.pop(msg.note)
                duration = current_time - start_time

                # Check if this note off ends the current chord
                if all(n not in note_start_times for n, v in current_chord):
                    chord_notes = [n for n, v in current_chord]
                    if chord_notes:
                        # Use the velocity of the first note for the whole chord
                        chords.append((chord_notes, current_chord[0][1], duration))
                    current_chord = []

    # Format the chords into a string and write to a file
    chords_string = '\n'.join([' '.join(map(str, chord[0])) + ',' + str(chord[1]) + ',' + str(chord[2]) for chord in chords])

    with open('midi_chords_formatted.txt', 'w') as file:
        file.write(chords_string)

    return chords

# Example usage
midi_path = '/content/Queen_midi.midi'
chords = midi_to_notes(midi_path)

# Print the chords for review
for chord in chords:
    print(chord)


([72, 74], 121, 1718.75)
([70], 121, 250.0)
([72], 122, 234.375)
([74, 73], 117, 1390.625)
([74], 123, 244.79166666662786)
([75], 123, 250.0)
([77, 75], 122, 250.0)
([74], 120, 218.75)
([72], 120, 807.2916666666279)
([72, 74, 75], 120, 239.58333333325572)
([77, 75], 121, 208.33333333337214)
([74, 72], 120, 1192.7083333332557)
([73, 74], 122, 197.91666666662786)
([74, 72], 122, 119.79166666662786)
([70], 120, 109.375)
([72, 74], 118, 151.04166666662786)
([77], 123, 197.91666666662786)
([80, 81], 122, 348.95833333337214)
([79, 78], 120, 104.16666666674428)
([79], 122, 125.0)
([82], 123, 208.33333333337214)
([82], 122, 114.58333333337214)
([82], 122, 489.58333333337214)
([81, 82], 120, 369.79166666662786)
([79], 123, 223.95833333337214)
([74, 75, 72], 123, 1270.8333333333721)
([78, 79], 123, 1489.5833333332557)
([77, 79, 80, 79], 123, 1484.375)
([79], 123, 218.75)
([79, 80], 123, 348.95833333337214)
([79], 123, 171.875)
([79, 77], 123, 145.83333333337214)
([77, 70], 123, 114.5833333333721